In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import math
from torchvision import datasets, transforms
import torch.nn.functional as F

In [3]:
torch.manual_seed(0)
class Net(torch.nn.Module):
    def __init__(self, dropout1, dropout2):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 5, 1)
        torch.nn.init.xavier_normal(self.conv1.weight, gain=np.sqrt(2))
        torch.nn.init.constant(self.conv1.bias, 0.1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        torch.nn.init.xavier_normal(self.conv2.weight, gain=np.sqrt(2))
        torch.nn.init.constant(self.conv2.bias, 0.1)
        self.conv3 = torch.nn.Conv2d(64, 128, 3, 1)
        torch.nn.init.xavier_normal(self.conv3.weight, gain=np.sqrt(2))
        torch.nn.init.constant(self.conv3.bias, 0.1)
        self.fc1 = torch.nn.Linear(73728, 1024)
        self.fc2 = torch.nn.Linear(1024, 1024)
        self.fc3 = torch.nn.Linear(1024, 10)
        self.dropout1 = torch.nn.Dropout(dropout1)
        self.dropout2 = torch.nn.Dropout(dropout2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        
        x = self.conv2(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        
        x = self.conv3(x)
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
def train(model, train_loader, optimizer, epoch, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 16 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

In [4]:
def main(lr=0.01, dropout1= 0.25, dropout2 = 0.25, weight_decay=5e-5):
    epochs = 1
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    kwargs = {'num_workers': 1, 'pin_memory': True}
    data, data_valid, data_test, _ = torch.utils.data.random_split(datasets.SVHN('./data', split='train',transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) \
                                     ,target_transform=None, download=False), [10000, 500, 2000, 60757])
    train_loader = torch.utils.data.DataLoader(data, batch_size=16, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(data_valid, batch_size=16, shuffle=True, **kwargs)
    finaltest_loader = torch.utils.data.DataLoader(data_test, batch_size=2000, shuffle=True, **kwargs)
    model = Net(dropout1, dropout2).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr = lr, momentum = 0.90, weight_decay = weight_decay)
    #optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay, amsgrad=False)
    print(train_loader)
    for epoch in range(1, epochs + 1):
        train(model, train_loader, optimizer, epoch, device)
        test(model, test_loader, device)
        optimizer.step()
    return test(model, finaltest_loader, device)

In [5]:
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import cholesky
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, Matern, RBF

noise = 1e-5
def kernel(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1 ** 2, 1).reshape(-1, 1) + np.sum(X2 ** 2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f ** 2 * np.exp(-0.5 / l ** 2 * sqdist)

In [6]:
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.0095):
    #print(str.format('current EI X is {}',X))
    mu, sigma = gpr.predict(X, return_std=True)
    #print(str.format('mu is {}',mu))
    #print(str.format('sigma is {}',sigma))
    mu_sample = gpr.predict(X_sample)
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu_sample)
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    #print(str.format('ei is {}',ei))
    return ei

In [7]:
#Reference taken from http://krasserm.github.io/2018/03/21/bayesian-optimization/?fbclid=IwAR3oWQVDeukrs63SM8P-nzWa7fht0TfYPzm1Tooncgc7xaasq-CGXmpxuhQ
def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None

    def min_obj(X):
        # Minimization objective is the negative acquisition function
        #print(str.format('min obj x is {}', X))
        return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)

    # Find the best optimum by starting from n_restart different random points.
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(100, dim)):
        #print(str.format('current x0 is {}',x0.reshape(1,-1)))
        boundstuple = None
        bl = []
        for i in range(bounds.shape[0]):
            bl.append(tuple(bounds[i,:]))
        boundstuple = tuple(bl)
        res = minimize(min_obj, x0=x0, bounds=boundstuple, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun[0]
            min_x = res.x
# Reference for random search taken from https://github.com/thuijskens/bayesian-optimization/blob/master/python/gp.py
#     x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(25, dim))
#     ei = -1 * expected_improvement(x_random.reshape(-1,dim), X_sample, Y_sample, gpr)
#     next_sample = x_random[np.argmax(ei), :]

    return min_x.reshape(-1, 1)

In [8]:
# def propose_location(acquisition, X_sample, Y_sample, gpr, bounds, n_restarts=25):
#     dim = X_sample.shape[1]
#     min_val = 1
#     min_x = None

#     def min_obj(X):
#         # Minimization objective is the negative acquisition function
#         print(str.format('min obj x is {}', X.reshape(-1,dim)))
#         print(str.format('min obj x shape is {}', X.reshape(-1,dim).shape))
#         return -acquisition(X.reshape(-1, dim), X_sample, Y_sample, gpr)

#     # Find the best optimum by starting from n_restart different random points.
#     for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(100, dim)):
#         #print(str.format('current x0 is {}',x0.reshape(1,-1)))
#         boundstuple = None
#         bl = []
#         for i in range(bounds.shape[0]):
#             if i == 0:
#                 bl.append(tuple(np.power([10,10],bounds[i,:])))
#             elif i == 3:
#                 bl.append(tuple(np.exp(bounds[i,:])))
#             else:
#                 bl.append(tuple(bounds[i,:]))
#         boundstuple = tuple(bl)
#         x0[0] = np.power([10],x0[0])
#         x0[3] = np.exp(x0[3])
#         res = minimize(min_obj, x0=np.squeeze(x0), bounds=boundstuple, method='L-BFGS-B')
#         print(res.x)
#         print(res.fun)
#         if res.fun < min_val:
#             print(res.x)
#             print(res.fun)
#             min_val = res.fun[0]
#             min_x = res.x

#     return min_x.reshape(-1, 1)

In [9]:
def gpHandler(X_init, Y_init, bounds,d=1):
    
    
    # Gaussian process with Mat??rn kernel as surrogate model
    
    
    m52 = ConstantKernel(1.0) * RBF(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0))
    gpr = GaussianProcessRegressor(kernel=m52, alpha=noise ** 2, normalize_y = True,n_restarts_optimizer=10)
    X_sample = X_init
    Y_sample = Y_init
    n_iter = 10
    gpr.fit(X_sample, Y_sample)
    X_next = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds)
    #plot_approximation(gpr,X_sample,Y_sample,bounds)
    print(X_next)
    return X_next.T

In [10]:
def scaler(x):
    #x[0] = np.exp(-int(5*x[0])-1)
    x[0] = np.power([10],(-3*x[0])-1)
    x[3] = np.power([10],(-4*x[3])-4)
    return x

In [11]:
def plot_approximation(gpr, X_sample, Y_sample,bounds, X_next=None, show_legend=False):
    X = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(100, 4))
    mu = np.zeros((100,1))
    std = np.zeros((100,1))
    counter=0
    for x0 in X:
        
        mu[counter,:],std[counter,:] = gpr.predict(x0.reshape(1,-1), return_std=True)
        counter = counter + 1
        #mu[counter,:], std[counter,:] = m.T, s.T 
    print(mu.shape)
    print(mu)
    plt.fill_between(X[:,0].ravel(), 
                     mu[:,0].ravel() + 1.96 * std[:,0], 
                     mu[:,0].ravel() - 1.96 * std[:,0], 
                     alpha=0.1) 
    X1 = X[:,0]
    X_sample1 = X_sample[:,0]
    #plt.plot(X, Y, 'y--', lw=1, label='Noise-free objective')
    plt.plot(X1, mu[:,0], 'b-', lw=1, label='Surrogate function')
    plt.plot(X_sample1, Y_sample, 'kx', mew=3, label='Noisy samples')
    if X_next:
        plt.axvline(x=X_next, ls='--', c='k', lw=1)
    if show_legend:
        plt.legend()
    plt.show()

In [12]:
bounds = np.array([[0, 1.0], [0, 1],[0, 1], [0, 1]]).reshape(-1,2)
#bounds = np.array([[0,1.0],[2.39340372e-01,2.39340372e-01],[1.74765370e-01,1.74765370e-01],[0.07791758,0.07791758]])
#bounds = np.array([[0, 0.02],[0, 0.5], [0, 0.5]]).reshape(-1,2)
X_latest = np.random.uniform(bounds[:,0], bounds[:,1], (1, 4))
#X_latest=np.array([0.01,0.25,0.25,0.05]).reshape(1,4)
#X_bar = scaler(X_latest.reshape(4,))
#print(X_bar)
#X_lat = X_latest.reshape(4,)
X_bar = np.array(X_latest,copy=True)
X_bar = scaler(X_bar.reshape(4,))
print(str.format('Parameters after scaling are {}',X_bar))
#loss = np.array([main(X_bar[0], X_bar[1], X_bar[2], X_bar[3])]).reshape(-1,1)
loss = np.array([main(X_bar[0], X_bar[1], X_bar[2],X_bar[3])]).reshape(-1,1)
hyperparameters = X_latest
print(loss)
for i in range(13):
    X_latest=(gpHandler(hyperparameters,loss, bounds, 4))
    #X_latest = np.random.uniform(bounds[:,0], bounds[:,1], (1, 4))
    hyperparameters =np.vstack((hyperparameters, X_latest))
    X_bar = np.array(X_latest,copy=True)
    X_bar = scaler(X_bar.reshape(4,))
    print(str.format('Parameters after scaling are {}',X_bar))
    #X_latest=scaler(X_latest.reshape(4,))
    #2.39340372e-01,1.74765370e-01,4.87898721e-05
    loss_latest=main(X_bar[0],X_bar[1],X_bar[2],X_bar[3])
    if math.isnan(loss_latest):
        loss_latest = np.array([3]).reshape(-1,1)
    else:
        loss_latest = np.array([loss_latest]).reshape(-1,1)
        
    #loss = np.vstack((loss, np.array(main(X_latest[0],X_latest[1],X_latest[2],X_latest[3])).reshape(-1,1)))
    loss = np.vstack((loss, loss_latest))
                     

Parameters after scaling are [2.05525108e-04 7.67050550e-01 1.14296877e-01 1.19650291e-07]


/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if __name__ == '__main__':
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  # Remove the CWD from sys.path while we load stuff.
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if sys.path[0] ==

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.306254
Train Epoch: 1 [256/10000 (3%)]	Loss: 2.275999
Train Epoch: 1 [512/10000 (5%)]	Loss: 2.330029
Train Epoch: 1 [768/10000 (8%)]	Loss: 2.270215
Train Epoch: 1 [1024/10000 (10%)]	Loss: 2.262439
Train Epoch: 1 [1280/10000 (13%)]	Loss: 2.138591
Train Epoch: 1 [1536/10000 (15%)]	Loss: 2.031203
Train Epoch: 1 [1792/10000 (18%)]	Loss: 2.215308
Train Epoch: 1 [2048/10000 (20%)]	Loss: 2.226545
Train Epoch: 1 [2304/10000 (23%)]	Loss: 2.245781
Train Epoch: 1 [2560/10000 (26%)]	Loss: 2.065126
Train Epoch: 1 [2816/10000 (28%)]	Loss: 2.199734
Train Epoch: 1 [3072/10000 (31%)]	Loss: 2.437139
Train Epoch: 1 [3328/10000 (33%)]	Loss: 2.257808
Train Epoch: 1 [3584/10000 (36%)]	Loss: 2.364769
Train Epoch: 1 [3840/10000 (38%)]	Loss: 2.353490
Train Epoch: 1 [4096/10000 (41%)]	Loss: 2.229433
Train Epoch: 1 [4352/10000 (44%)]	Loss: 2.253492
Train Epoch: 1 [4608/10000 (46%)]	Loss: 2.234202
Train Epoch: 1 [4864/10000 (49%)]	Loss: 2.268662
Train Epoch: 1 [5120/10000 (51

/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if __name__ == '__main__':
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  # Remove the CWD from sys.path while we load stuff.
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if sys.path[0] ==

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.339073
Train Epoch: 1 [256/10000 (3%)]	Loss: 2.244833
Train Epoch: 1 [512/10000 (5%)]	Loss: 2.498640
Train Epoch: 1 [768/10000 (8%)]	Loss: 2.215632
Train Epoch: 1 [1024/10000 (10%)]	Loss: 2.090957
Train Epoch: 1 [1280/10000 (13%)]	Loss: 2.198137
Train Epoch: 1 [1536/10000 (15%)]	Loss: 2.200624
Train Epoch: 1 [1792/10000 (18%)]	Loss: 2.350876
Train Epoch: 1 [2048/10000 (20%)]	Loss: 2.162290
Train Epoch: 1 [2304/10000 (23%)]	Loss: 2.141963
Train Epoch: 1 [2560/10000 (26%)]	Loss: 2.320350
Train Epoch: 1 [2816/10000 (28%)]	Loss: 2.168784
Train Epoch: 1 [3072/10000 (31%)]	Loss: 2.330093
Train Epoch: 1 [3328/10000 (33%)]	Loss: 2.234236
Train Epoch: 1 [3584/10000 (36%)]	Loss: 2.393271
Train Epoch: 1 [3840/10000 (38%)]	Loss: 2.332318
Train Epoch: 1 [4096/10000 (41%)]	Loss: 2.305269
Train Epoch: 1 [4352/10000 (44%)]	Loss: 2.070691
Train Epoch: 1 [4608/10000 (46%)]	Loss: 2.027864
Train Epoch: 1 [4864/10000 (49%)]	Loss: 2.175123
Train Epoch: 1 [5120/10000 (51

...

mu is [[2.12200448]]
mu is [[2.12200449]]
mu is [[2.03007979]]
mu is [[2.03007979]]
mu is [[2.03007979]]
mu is [[2.03007979]]
mu is [[2.03007979]]
mu is [[2.12011135]]
mu is [[2.12011135]]
mu is [[2.12011135]]
mu is [[2.12011134]]
mu is [[2.12011135]]
mu is [[2.03749349]]
mu is [[2.03749349]]
mu is [[2.03749349]]
mu is [[2.03749349]]
mu is [[2.03749349]]
mu is [[2.12964945]]
mu is [[2.12964945]]
mu is [[2.12964946]]
mu is [[2.12964945]]
mu is [[2.12964945]]
mu is [[2.13910289]]
mu is [[2.13910289]]
mu is [[2.13910289]]
mu is [[2.13910289]]
mu is [[2.13910289]]
mu is [[2.1403054]]
mu is [[2.1403054]]
mu is [[2.14030541]]
mu is [[2.1403054]]
mu is [[2.1403054]]
mu is [[2.14960144]]
mu is [[2.14960144]]
mu is [[2.14960144]]
mu is [[2.14960144]]
mu is [[2.14960144]]
mu is [[2.14426243]]
mu is [[2.14426243]]
mu is [[2.14426244]]
mu is [[2.14426243]]
mu is [[2.14426243]]
mu is [[2.14321607]]
mu is [[2.14321607]]
mu is [[2.14321608]]
mu is [[2.14321607]]
mu is [[2.14321608]]
mu is [[2.1418439

mu is [[2.12740981]]
mu is [[2.12740981]]
mu is [[2.12740981]]
mu is [[2.12740981]]
mu is [[2.12740981]]
mu is [[2.12648638]]
mu is [[2.12648638]]
mu is [[2.12648639]]
mu is [[2.12648638]]
mu is [[2.12648638]]
mu is [[2.12441942]]
mu is [[2.12441942]]
mu is [[2.12441943]]
mu is [[2.12441942]]
mu is [[2.12441942]]
mu is [[2.13271742]]
mu is [[2.13271741]]
mu is [[2.13271742]]
mu is [[2.13271741]]
mu is [[2.13271742]]
mu is [[2.1350617]]
mu is [[2.1350617]]
mu is [[2.1350617]]
mu is [[2.1350617]]
mu is [[2.1350617]]
mu is [[2.10167632]]
mu is [[2.10167632]]
mu is [[2.10167633]]
mu is [[2.10167632]]
mu is [[2.10167633]]
mu is [[2.13707175]]
mu is [[2.13707175]]
mu is [[2.13707176]]
mu is [[2.13707175]]
mu is [[2.13707175]]
mu is [[2.11153484]]
mu is [[2.11153484]]
mu is [[2.11153485]]
mu is [[2.11153484]]
mu is [[2.11153484]]
mu is [[2.13299671]]
mu is [[2.13299671]]
mu is [[2.13299672]]
mu is [[2.13299671]]
mu is [[2.13299672]]
mu is [[2.13795281]]
mu is [[2.1379528]]
mu is [[2.13795281]

mu is [[2.02536817]]
mu is [[2.03429288]]
mu is [[2.03429288]]
mu is [[2.03429288]]
mu is [[2.03429288]]
mu is [[2.03429288]]
mu is [[2.03458445]]
mu is [[2.03458445]]
mu is [[2.03458445]]
mu is [[2.03458445]]
mu is [[2.03458445]]
mu is [[2.03558057]]
mu is [[2.03558057]]
mu is [[2.03558057]]
mu is [[2.03558057]]
mu is [[2.03558057]]
mu is [[2.03561258]]
mu is [[2.03561258]]
mu is [[2.03561258]]
mu is [[2.03561258]]
mu is [[2.03561258]]
mu is [[2.03575143]]
mu is [[2.03575143]]
mu is [[2.03575143]]
mu is [[2.03575143]]
mu is [[2.03575143]]
mu is [[2.03647472]]
mu is [[2.03647472]]
mu is [[2.03647472]]
mu is [[2.03647472]]
mu is [[2.03647472]]
mu is [[2.04080806]]
mu is [[2.04080806]]
mu is [[2.04080806]]
mu is [[2.04080806]]
mu is [[2.04080806]]
mu is [[2.03886111]]
mu is [[2.03886111]]
mu is [[2.03886111]]
mu is [[2.03886111]]
mu is [[2.03886111]]
mu is [[2.05023006]]
mu is [[2.05023006]]
mu is [[2.05023006]]
mu is [[2.05023006]]
mu is [[2.05023006]]
mu is [[2.03908913]]
mu is [[2.039

/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if __name__ == '__main__':
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  # Remove the CWD from sys.path while we load stuff.
/home/spatri/anaconda3/envs/workspace/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  if sys.path[0] ==

Train Epoch: 1 [0/10000 (0%)]	Loss: 2.303225
Train Epoch: 1 [256/10000 (3%)]	Loss: 2.384926
Train Epoch: 1 [512/10000 (5%)]	Loss: 2.291605
Train Epoch: 1 [768/10000 (8%)]	Loss: 2.388402
Train Epoch: 1 [1024/10000 (10%)]	Loss: 2.393984
Train Epoch: 1 [1280/10000 (13%)]	Loss: 2.309480
Train Epoch: 1 [1536/10000 (15%)]	Loss: 2.371728
Train Epoch: 1 [1792/10000 (18%)]	Loss: 2.209027
Train Epoch: 1 [2048/10000 (20%)]	Loss: 2.401582
Train Epoch: 1 [2304/10000 (23%)]	Loss: 2.121862
Train Epoch: 1 [2560/10000 (26%)]	Loss: 2.294615
Train Epoch: 1 [2816/10000 (28%)]	Loss: 2.168745
Train Epoch: 1 [3072/10000 (31%)]	Loss: 2.190681
Train Epoch: 1 [3328/10000 (33%)]	Loss: 2.321695
Train Epoch: 1 [3584/10000 (36%)]	Loss: 2.154866
Train Epoch: 1 [3840/10000 (38%)]	Loss: 2.346173
Train Epoch: 1 [4096/10000 (41%)]	Loss: 2.202667
Train Epoch: 1 [4352/10000 (44%)]	Loss: 2.105247
Train Epoch: 1 [4608/10000 (46%)]	Loss: 2.186435
Train Epoch: 1 [4864/10000 (49%)]	Loss: 2.062751
Train Epoch: 1 [5120/10000 (51

In [ ]:
print(loss)

In [ ]:
# #bounds = np.array([[0, 1.0], [0, 1.0],[0, 0.5], [0, 0.5]]).reshape(-1,2)
# #bounds = np.array([[0, 0.02],[0, 0.5], [0, 0.5]]).reshape(-1,2)
# loss = np.array([3]).reshape(1,1)
# bounds = np.array([[-4, 0],[0, 0.5], [0, 0.5],[-3,-8]]).reshape(-1,2)
# hyperparameters = np.random.uniform(bounds[:,0], bounds[:,1], (1, 4))
# for i in range(1):
    
#     X_latest = np.random.uniform(bounds[:,0], bounds[:,1], (1, 4))
#     #X_latest = np.random.uniform(bounds[:,0], bounds[:,1], (1, 3))
#     #X_latest=np.array([0.01,0.25,0.25]).reshape(1,3)
#     #X_bar = np.array(X_latest,copy=True)
#     X_bar = scaler(X_latest.reshape(4,))
#     #print(X_bar)
#     #X_lat = X_latest.reshape(3,)
#     print(str.format('Parameters before scaling are {}',X_latest))
#     print(str.format('Parameters after scaling are {}',X_bar))
#     loss = np.vstack((loss,np.array([main(X_bar[0], X_bar[1], X_bar[2], X_bar[3])]).reshape(-1,1)))
#     #loss = np.array([main(X_lat[0], X_lat[1], X_lat[2])]).reshape(-1,1)
#     #hyperparameters = X_bar.reshape(1,4)
#     hyperparameters = np.vstack((hyperparameters, X_bar.reshape(1,4)))
#     print(loss)
# hyperparameters = np.delete(hyperparameters,0,0)
# loss = np.delete(loss,0,0)
# for i in range(14):
#     X_latest=(gpHandler(hyperparameters,loss, bounds, 4))
#     hyperparameters =np.vstack((hyperparameters, X_latest))
#     X_latest = X_latest.reshape(4,)
#     #X_latest=scaler(X_latest.reshape(4,))
#     #X_bar = scaler(X_latest.reshape(4,))
#     print(str.format('Parameters are {}',X_latest))
#     #print(str.format('Parameters after scaling are {}',X_bar))
#     loss_latest=main(X_latest[0],X_latest[1],X_latest[2],X_latest[3])
#     if math.isnan(loss_latest):
#         loss_latest = np.array([3]).reshape(-1,1)
#     else:
#         loss_latest = np.array([loss_latest]).reshape(-1,1)
        
#     #loss = np.vstack((loss, np.array(main(X_latest[0],X_latest[1],X_latest[2],X_latest[3])).reshape(-1,1)))
#     loss = np.vstack((loss, loss_latest))
                     

In [ ]:
print(loss)

In [ ]:
X_train = np.random.uniform(-1, 1, (1, 4))
print(X_train.shape)

In [ ]:
a = np.array([0.2,1,0.5,0.6])
print(a.shape)


In [ ]:
scaler(np.array([0.4370436 
,0.97170101
 ,0.82984924
 ,0.93905999]))

In [ ]:
bounds = np.array([[0, 1.0], [0, 1.0],[0, 0.5], [0, 0.5]]).reshape(-1,2)
print(bounds.shape)

In [ ]:
print(bounds)

In [ ]:
bounds = np.array([[-4, 0],[0, 0.5], [0, 0.5],[-3,-8]]).reshape(-1,2)
X_latest = np.random.uniform(bounds[:,0], bounds[:,1], (1, 4))
print(X_latest)
#X_latest=np.array([0.01,0.25,0.25]).reshape(1,3)
X_bar = scaler(X_latest.reshape(4,))
print(X_bar)

In [ ]:
t = np.linspace(1, 14, num=14)
print(loss.shape)
print(t.shape)
#plt.plot(t, loss)
accuracy = [70, 74, 82, 85, 85, 86, 18, 6, 35, 69, 86, 68, 18, 84]
#plt.plot(t, accuracy)
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Loss', color=color)
ax1.plot(t, loss, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx() 
color = 'tab:blue'
ax2.set_ylabel('Accuracy', color=color)  # we already handled the x-label with ax1
ax2.plot(t, accuracy, color=color)
ax2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Accuracy and Loss vs iterations')
plt.show()

In [ ]:
%matplotlib inline
from scipy.spatial.distance import cdist
def plot_convergence(X_sample,Y_sample, n_init=2):
    plt.figure(figsize=(12, 3))

    x = X_sample[n_init:].ravel()
    y = Y_sample[n_init:].ravel()
    r = range(1, len(y)+1)
    r1 = range(1, X_sample.shape[0])
    
    x_neighbor_dist = [np.abs(a-b) for a, b in zip(x, x[1:])]
    l1 = []
    print(X_sample[2,:].shape)
    for i in range(1,X_sample.shape[0]):
        l1.append(np.linalg.norm(X_sample[i-1,:].reshape(-1,1)-X_sample[i,:].reshape(-1,1)))
    x_neighbor_dist = np.array(l1)
    print(x_neighbor_dist.shape)
    y_max_watermark = np.maximum.accumulate(y)
    x_max_watermark = np.maximum.accumulate(x)
    
    plt.subplot(1, 2, 1)
    plt.plot(r1, x_neighbor_dist, 'bo-',label='Bayesian Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Distance')
    plt.title('Distance between consecutive x\'s')

    plt.subplot(1, 2, 2)
    plt.plot(r, y_max_watermark, 'ro-',label = 'Random Search Accuracy')
    plt.xlabel('Iteration')
    plt.ylabel('Accuracy')
    plt.title('Value of best selected sample')
    plt.legend(loc='lower right')
accuracy1 = np.array([19,69,72,77,76,80,34,81,20,19,74,79,80.5]) 
#accuracy = np.array([70,74,82,85,85,86,18,6,35,69,86,68,18,84])
accuracy = hyperparameters
hyperparameters1 = np.array([[2.31500899e-01, 3.61065847e-01, 3.69937984e-01, 9.45312040e-01],
 [8.71323986e-01, 3.71091044e-01, 3.99542191e-01, 5.33367902e-03],
  [8.33454648e-01, 2.00370759e-01, 1.49660999e-01, 3.29171196e-01],
  [7.22220286e-01, 4.83327533e-01, 2.13163736e-01, 8.78210250e-01],
  [4.90731253e-01, 1.64544117e-01, 4.13883524e-01, 8.98052127e-01],
  [5.24546342e-01, 4.71271316e-01, 1.66267121e-01, 7.34976659e-01],
  [1.28806481e-01, 3.77740845e-01, 3.86714457e-01, 9.69652831e-01],
  [2.00830338e-01, 5.13755629e-03, 3.79561534e-01, 1.72460390e-01],
  [1.86811942e-01, 5.25170041e-02, 3.68392378e-01, 1.54489818e-01],
  [2.13563045e-01, 5.67124702e-02, 3.70542631e-01, 1.87915623e-01],
  [6.28669132e-01, 4.80973743e-01, 2.18346015e-01, 1.97632493e-01],
  [9.48268630e-01, 4.82577717e-04, 2.82358787e-01, 4.09744367e-01],
  [7.91967436e-02, 2.45379371e-01, 2.95664116e-01, 7.58209610e-02],
  [7.58229923e-01, 2.23483844e-01, 3.93908302e-01, 6.90523310e-01]])
print(hyperparameters1.shape)
plot_convergence(hyperparameters1[:-3],accuracy1[:-1])



